# Pydantic

- can work with data in an OOP manner
- fields and methods
- validate data 
- output json data

pydantic model created by inheriting from BaseModel

## Start without pydantic 

In [1]:
# defined a class Person (blueprint)
class Person:
    def __init__(self, name, gender, age):
        self.name = name 
        self.gender = gender
        self.age = age 

# instantiated an instance from the Person class
person1 = Person(name = "Kokchun", age = 34, gender = "M")
person1



In [3]:
# attribute in this person1 instance
person1.name, person1.gender, person1.age

('Kokchun', 'M', 34)

In [6]:
# crazy but works
person2 = Person(name = 312+8, gender = True, age = "minus tio")
person2.name, person2.gender, person2.age

(320, True, 'minus tio')

what if we type hint?

In [9]:
# used type hinting
class Person:
    def __init__(self, name: str, gender: str, age: int): 
        self.name = name 
        self.gender = gender
        self.age = age 

person3 = Person("Gauss", "M", 70)
person3

In [10]:
# crazy again and it works
person4 = Person(3.1415, 2.714, "fisk")
person4.age


'fisk'

### validate our Person class 

In [18]:
class Person:
    def __init__(self, name: str, gender: str, age: int): 
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        
        self.gender = gender
        self.age = age 

try:
    Person(name = 532, gender = 3, age = "hare")
except TypeError as err:
    print(err)


name must be of type str not <class 'int'>


In [19]:
class Person:
    def __init__(self, name: str, gender: str, age: int): 
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        
        self.gender = gender
        self.age = age 
    
    # gettter
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value: int):
        if not isinstance(value, int):
            raise TypeError(f"age must be of type int not {type(age)} that you have provided")
        if value < 0 or value > 125:
            raise ValueError(f"age must be between 0 and 125, not {value} that you provided")
        
        self._age = value
    

person5 = Person(name = "Bella", gender = "F", age = 3)
person5.age


3

In [21]:
try:
    Person(name = "Bella", gender = "F", age = -3)
except ValueError as err:
    print(err)

age must be between 0 and 125, not -3 that you provided


In [22]:
try:
    Person(name = "Bella", gender = "F", age = 3121)
except ValueError as err:
    print(err)

age must be between 0 and 125, not 3121 that you provided


In [24]:
try:
    Person(name = "Bella", gender = "F", age = "femtioåtta")
except NameError as err:
    print(err)

name 'age' is not defined


## work smarter with pydantic

In [25]:
from pydantic import BaseModel

# inherits from BaseModel -> makes it into pydantic model, but still a normal python class
class Person(BaseModel):
    name: str 
    gender: str 
    age: int

person6 = Person(name = "Magnus", gender = "M", age = 35)
person6

Person(name='Magnus', gender='M', age=35)

In [27]:
person6.name, person6.age

('Magnus', 35)

In [28]:
person6.age = 36
person6.age

36

In [29]:
person6

Person(name='Magnus', gender='M', age=36)

In [32]:
from pydantic import ValidationError
try:
    Person(name = 3.1415, gender=2.714, age = -5)
except ValidationError as err:
    print(err)


2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.1415, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


In [33]:
try:
    Person(name = "Sofia", gender=2.714, age = -5)
except ValidationError as err:
    print(err)

1 validation error for Person
gender
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


bad, we have negative age

In [34]:
Person(name = "Sofia", gender="F", age = -5)

Person(name='Sofia', gender='F', age=-5)

### Fix person class with age validation

In [39]:
from pydantic import Field


class Person(BaseModel):
    name: str
    gender: str
    age: int = Field(gt=-1, lt=126)


try:
    Person(name = "Sofia", gender="F", age = 126)
except ValidationError as err:
    print(err)

1 validation error for Person
age
  Input should be less than 126 [type=less_than, input_value=126, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than


In [ ]:
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt=-1, lt=126)

try:
    Person(name = "Sofia", gender="Female", age = 26)
except ValidationError as err:
    print(err)



1 validation error for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Female', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error


Person(name='Sofia', gender='F', age=26)

In [43]:
person7 = Person(name = "Sofia", gender="F", age = 26)
person7

Person(name='Sofia', gender='F', age=26)

make the instance into a dictionary

In [ ]:
person7.model_dump()

{'name': 'Sofia', 'gender': 'F', 'age': 26}

In [ ]:
import json
with open("person.json", "w") as f:
    json.dump(person7.model_dump(), f)

person7.model_dump()

make instance into a json string - serialization

In [ ]:
person7.model_dump_json()

'{"name":"Sofia","gender":"F","age":26}'

deserialize a json string back into an instance

In [ ]:
with open("person.json", "r") as file:
    json_data = file.read()

Person.model_validate_json(json_data)